In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('variables.csv')

In [ ]:
df = df.drop(columns=['Epic_week'])
df

Next, we discuss the autoregressive model. Autoregression is actually a special linear regression model in which the independent variable is the lagged value of the dependent variable. In this study, we also consider the lagged values of other exogenous variables. This is for the subsequent environmental epidemiological analysis

In [ ]:
import itertools
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

#### How to choose the optimal (p,q)?  ACF+PACF+AIC or BIC

In [ ]:
# for col in df.columns:
#     fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,4))
#     plot_acf(df[col], ax=ax[0], lags=30)
#     plot_pacf(df[col], ax=ax[1], lags=30)
#     ax[0].set(title='Autocorrelation for ({})'.format(col))
#     ax[1].set(title='Partial Autocorrelation for ({})'.format(col))
#     plt.show()

### Autoregression with exogenous variables model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
n_lag = 4
n_steps = 8
# Create lagged variables for the deseas count, temperature, and rainfall
columns = list(df.columns)

for col in columns:
    for i in range(1, n_lag + 1):
        df[f"{col}_lag_{i}"] = df[col].shift(i)

# Drop rows with NaN values generated by shifting
df = df.dropna()

# Only keep the target column when there is no lag
df = df.drop(df.columns[1:50], axis=1)

In [ ]:
y = df['Infectious and Parasitic Diseases'].shift(-n_steps+1)
y = y.dropna()
if n_steps != 1:
  df = df.iloc[:-n_steps+1, :]

X = df.drop('Infectious and Parasitic Diseases', axis=1)

In [ ]:
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [ ]:
# when you output the regression table. dont run this 2 lines of code.
X = np.array(X)
y = np.array(y)

In [ ]:
# Cross validation.
train_size = int(len(X) * 0.7)
test_ind = range(train_size, len(X))

y_pred_store = np.array([None] * len(X))
y_test_array = np.array([])
y_pred_array = np.array([])

for forecast_ind in test_ind:
    train_ind = list(range(0, forecast_ind))
    X_train = X[train_ind]
    X_test = X[forecast_ind].reshape(1, -1)
    y_train = y[train_ind]
    y_test = y[forecast_ind]

    # Z-score
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Create the Linear Regression model
    model = LinearRegression()

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_store[forecast_ind] = y_pred
    y_test_array = np.append(y_test_array, y_test)
    y_pred_array = np.append(y_pred_array, y_pred)



In [ ]:
# Calculate the mean squared error on training and test sets

mse = mean_squared_error(y_test_array, y_pred_array)
print('Mean Squared Error for {} step: {}'.format(n_steps ,mse))

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test_array, y_pred_array)
print('Mean Absolute Percentage Error for {} step: {:.2f}%'.format(n_steps,mape))

In [ ]:
plt.figure(figsize=(9, 4))

# Set the x-axis limits
plt.xlim(-30, len(X)+40)

# Shade the left half of the figure in light blue
first_test_index = test_ind[0]  # Get the index of the first test observation
plt.axvspan(-30, first_test_index, facecolor='lightblue', alpha=0.5)

plt.plot(y, label='Observed', linestyle='-', linewidth=0.6, alpha=1, color='black')
plt.scatter(range(len(y_pred_store)), y_pred_store, label=F'AREV {n_steps} weeks ahead forecast', marker='o', s=10, alpha=1, color='brown')

plt.title(' Infectious and Parasitic Diseases Prediction (step:{})'.format(n_steps))
plt.xlabel('Epidemiological Week (Index in the forecast set)')
plt.ylabel('IPDs ED Admission')



plt.legend(loc='upper left', frameon=False)
plt.show()


### Autoregression with exogenous variables model (lasso)

In [ ]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
y_pred_store = np.array([None] * len(X))
y_test_array = np.array([])
y_pred_array = np.array([])

for forecast_ind in test_ind:
    train_ind = list(range(0, forecast_ind))
    X_train = X[train_ind]
    X_test = X[forecast_ind].reshape(1, -1)
    y_train = y[train_ind]
    y_test = y[forecast_ind]

    # Z-score
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Create the LassoCV model
    tscv = TimeSeriesSplit(n_splits=5)
    # alphas = np.logspace(-4, 0, 100)  # You can adjust the range and number of alpha values to search
    lasso_cv = LassoCV(alphas=None, cv=tscv, random_state=42)

    lasso_cv.fit(X_train, y_train)

    optimal_alpha = lasso_cv.alpha_

    # Create the Lasso model with the optimal alpha value
    model = Lasso(alpha=optimal_alpha)

    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_store[forecast_ind] = y_pred
    y_test_array = np.append(y_test_array, y_test)
    y_pred_array = np.append(y_pred_array, y_pred)

In [ ]:
# Calculate the mean squared error on training and test sets

mse = mean_squared_error(y_test_array, y_pred_array)
print('Mean Squared Error for {} step: {}'.format(n_steps ,mse))

mae = mean_absolute_error(y_test_array, y_pred_array)
print('Mean Absolute Error for {} step: {}'.format(n_steps ,mae))

In [ ]:
mape = mean_absolute_percentage_error(y_test_array, y_pred_array)
print('Mean Absolute Percentage Error for {} step: {:.2f}%'.format(n_steps,mape))

In [ ]:
def mean_absolute_scaled_error(y_true, y_pred):
    n = len(y_true)

    # Calculate absolute error of the forecasted values
    abs_err = np.abs(y_true[n_steps:] - y_pred[n_steps:])

    # Calculate the mean absolute error of the forecasted values
    mae = np.mean(abs_err)

    # Calculate the mean absolute error for the in-sample h-step naive forecast
    naive_forecast = y_true[:-n_steps]  # naive forecast shifts the series by h step
    mae_naive = np.mean(np.abs(y_true[n_steps:] - naive_forecast))

    # Calculate and return MASE
    mase = mae / mae_naive

    return mase

mase = mean_absolute_scaled_error(y_test_array, y_pred_array)
print('Mean Absolute Scaled Error for {} step: {:.2f}'.format(n_steps,mase))

In [ ]:
# 8 forecasting window. different size of samples.
# 1 week ahead: [0, 48, 100, 152, 204, 256, 309, 361, 413, 465, 517]
# 2 week ahead: [0, 47, 99, 151, 203, 255, 308, 360, 412, 464, 516]
# 3 week ahead: [0, 46, 98, 150, 202, 254, 307, 359, 411, 463, 515]
# 4 week ahead: [0, 45, 97, 149, 201, 253, 306, 358, 410, 462, 514]
# 5 week ahead: [0, 44, 96, 148, 200, 252, 305, 357, 409, 461, 513]
# 6 week ahead: [0, 43, 95, 147, 199, 251, 304, 356, 408, 460, 512]
# 7 week ahead: [0, 42, 94, 146, 198, 250, 303, 355, 407, 459, 511]
# 8 week ahead: [0, 41, 93, 145, 197, 249, 302, 354, 406, 458, 510]

In [ ]:
plt.figure(figsize=(9, 4))

# Set the x-axis limits
plt.xlim(-30, len(X)+40)

# Shade the left half of the figure in light blue
first_test_index = test_ind[0]  # Get the index of the first test observation
plt.axvspan(-30, first_test_index, facecolor='lightblue', alpha=0.4)

# Create tick locations and labels
tick_interval = 52
# origin data is [0, 52, 104, 156, 208, 260, 313, 365, 417, 469, 521]
tick_locations = np.array([0, 41, 93, 145, 197, 249, 302, 354, 406, 458, 510])
tick_label_locations = tick_locations[:-1] + tick_interval / 2
tick_labels = [f'{year:02d}' for year in range(9, 19)]

plt.plot(y, label='Observed', linestyle='-', linewidth=0.6, alpha=1, color='black')
plt.scatter(range(len(y_pred_store)), y_pred_store, label=F'AREV (LASSO) {n_steps} week ahead forecast', marker='o', s=10, alpha=1, color='darkorchid')


plt.xlabel('Year', fontsize=12)
plt.ylabel('IPDs ED Attendances', fontsize=12)

plt.xticks(tick_locations)
plt.gca().set_xticklabels([])
plt.gca().set_xticks(tick_label_locations, minor=True)
plt.gca().set_xticklabels(tick_labels, minor=True)
plt.gca().tick_params(axis='x', which='minor', length=0)


plt.legend(loc='upper left', frameon=False)
plt.savefig('LASSO8.tif', format='tif', dpi=400)
plt.show()


## Post-selection inference： 1-8 week 
####  Change the n_steps form 1 to 8 and terate the process 8 times. 

In [ ]:
!pip install stargazer
from stargazer.stargazer import Stargazer

In [ ]:
train_ind = list(range(0, len(X)))
X_train = X.iloc[train_ind]
y_train = y.iloc[train_ind]

# Z-score
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Create the LassoCV model
tscv = TimeSeriesSplit(n_splits=5)
# alphas = np.logspace(-4, 0, 100)  # You can adjust the range and number of alpha values to search
lasso_cv = LassoCV(alphas=None, cv=tscv, random_state=42)

lasso_cv.fit(X_train_scaled, y_train)

optimal_alpha = lasso_cv.alpha_

# Create the Lasso model with the optimal alpha value
model = Lasso(alpha=optimal_alpha)

model.fit(X_train_scaled, y_train)

# Make predictions
# y_pred = model.predict(X_test)
selected_features = model.coef_ != 0

In [ ]:
import statsmodels.api as sm

X_train_with_names = X_train.iloc[:, selected_features]
X_train_selected = sm.add_constant(X_train_with_names)
# X_test_selected = sm.add_constant(pd.DataFrame(X_test[:, selected_features])) # no test. only train. So this line is useless.

model = sm.OLS(y_train, X_train_selected)
results1 = model.fit()
print(results1.summary())

x1
x7
x14
x20
x24
x26
x30
x36 <0.05

Daily_Rainfall_Total_mm_lag_1:18; Minimum_Temperature_C_lag_2:-25; pm25_lag_4:21; co_lag_4:-26

In [ ]:
select_index = []
for i in range(len(selected_features)):
  if selected_features[i] == True:
    select_index.append(i)

In [ ]:
print(select_index[0])
print(select_index[6])
print(select_index[13])
print(select_index[19])
print(select_index[23])
print(select_index[25])
print(select_index[29])
print(select_index[35])

In [ ]:
print(df.columns[0+1], df.columns[23+1], df.columns[47+1], df.columns[76+1], df.columns[148+1], df.columns[161+1], df.columns[183+1], df.columns[199+1])

## Summary 
#### insight from analyses in 8 forecasting windows
week1: Daily_Rainfall_Total_mm_lag_1:18; Minimum_Temperature_C_lag_2:-25; pm25_lag_4:21; co_lag_4:-26

week2: Maximum_Temperature_C_lag_2：15.78; Minimum_Temperature_C_lag_1：-42; Max_Wind_Speed_kmh_lag_4:12; o3_lag_2:-18; co_lag_3:-26.8

week3： o3_lag_1: -17

week4: Mean_Wind_Speed_kmh_lag_1: -19; o3_lag_1: -22

week5: Minimum_Temperature_C_lag_1: -25 o3_lag_1:-24

week6: pm25_lag_4: 14

week7: o3_lag_1： -20

week8: Max_Wind_Speed_kmh_lag_4: -20; o3_lag_1: -22